In [ ]:
# default_exp data.tsloader

# TimeSeriesLoader
> Data Loader for Time Series data

In [ ]:
#hide
from nbdev import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#export
import copy
import logging
import random
from collections import defaultdict
from typing import Collection, Dict, List, Optional, Tuple, Union
from typing_extensions import Literal

import numpy as np
import pandas as pd
import torch as t
from fastcore.foundation import patch

from nixtla.data.tsdataset import TimeSeriesDataset

In [ ]:
#export
class TimeSeriesLoader(object):
    """
    DataLoader for Time Series data.
    
    Attributes
    ----------
    ts_dataset: TimeSeriesDataset
        Object of class TimeSeriesDataset.
    t_cols: list
        List of temporal variables (mask variables included).
    f_cols: list
        List of exogenous variables of the future.
    model: str
        Model to be used.
        One of ['nbeats', 'esrnn'].
    window_sampling_limit: int
        Max size of observations to consider, including output_size.
    input_size: int
        Size of the training sets.
    output_size: int
        Forecast horizon.
    idx_to_sample_freq: int
        Step size to construct windows.
        Ej. if idx_to_sample_freq=7, each 7 timestamps
        a window will be constructed.
    batch_size: int
        Number of samples considered in each iteration.
    complete_inputs: bool
        Whether consider only windows of length equals to input_size.
    shuffle: bool
        Shuffled batch.
        If False, batch size will be ignored and
        all windows will be used when training.
    len_sample_chunks: Optional[int] = None
        Size of complete windows.
        Only used for model = 'esrnn'!
        Default None, equls to input_size + ouput_size.
    n_series_per_batch: Optional[int] = None
        Number of time series per batch.
    verbose: bool = False
        Whether display informative messages.
    windows_size: int
        Size of the windows.
        For model='nbeats', window_size=input_size + output_size.
        For model='esrnn', window_size=len_sample_chunks.
    padding: Tuple[int, int]
        Tuple of left and right sizes of the padding.
        Used to pad the ts_tensor with 0.
        For model='nbeats', padding=(input_size, output_size).
        For model='esrnn', padding=(0, 0).
    sampleable_ts_idxs: np.ndarray
        Indexes of sampleable time series.
    n_sampleable_ts_idxs: int
        Number of sampleable time series.
    n_batches:
        Number of batches given conditions.
    
    Methods
    -------
    get_n_variables()
        Returns Tuple of number of exogenous and static variables.
    get_n_series()
        Returns number of time series.
    get_max_len()
        Returns max length of the time series.
    get_n_channels()
        Returns number of channels.
    get_frequency()
        Returns infered frequency.
    """
    def __init__(self,
                 ts_dataset: TimeSeriesDataset,
                 model: Literal['nbeats', 'esrnn'],
                 window_sampling_limit: int, 
                 input_size: int,
                 output_size: int,
                 idx_to_sample_freq: int,
                 batch_size: int,
                 complete_inputs: bool,
                 shuffle: bool,
                 len_sample_chunks: Optional[int] = None,
                 n_series_per_batch: Optional[int] = None,
                 verbose: bool = False) -> 'TimeSeriesLoader':
        """Instatiates loader for TimeSeriesDataset.
        
        Parameters
        ----------
        ts_dataset: TimeSeriesDataset
            Object of class TimeSeriesDataset.
        model: str
            Model to be used.
            One of ['nbeats', 'esrnn'].
        window_sampling_limit: int
            Max size of observations to consider, including output_size.
        input_size: int
            Size of the training sets.
        output_size: int
            Forecast horizon.
        idx_to_sample_freq: int
            Step size to construct windows.
            Ej. if idx_to_sample_freq=7, each 7 timestamps
            a window will be constructed.
        batch_size: int
            Number of samples considered in each iteration.
        complete_inputs: bool
            If complete_input=True
            return all windows which its ouput_size 
            has complete sample_mask and its input_size
            has complete available_mask.
            If complete_input=False
            returns all windows which its
            output_size has complete sample_mask. 
            This avoids leakage.
        shuffle: bool
            Shuffled batch.
            If False, batch size will be ignored and
            all windows will be used when training.
        len_sample_chunks: Optional[int] = None
            Size of complete windows.
            Only used for model = 'esrnn'!
            Default None, equls to input_size + ouput_size.
        n_series_per_batch: Optional[int] = None
            Number of time series per batch.
        verbose: bool = False
            Whether display informative messages.
        """
        # Dataloader attributes
        self.model = model
        self.window_sampling_limit = window_sampling_limit
        self.input_size = input_size
        self.output_size = output_size
        self.batch_size = batch_size
        self.complete_inputs = complete_inputs
        self.idx_to_sample_freq = idx_to_sample_freq
        self.ts_dataset = ts_dataset
        self.t_cols = self.ts_dataset.t_cols
        self.f_idxs = self.ts_dataset.f_idxs
        if n_series_per_batch is not None:
            self.n_series_per_batch = n_series_per_batch 
        else:
            self.n_series_per_batch = min(batch_size, self.ts_dataset.n_series)
            
        if len_sample_chunks is not None:
            if len_sample_chunks < self.input_size + self.output_size:
                raise Exception(f'Insufficient len of sample chunks {len_sample_chunks}')
            self.len_sample_chunks = len_sample_chunks
        else:
            self.len_sample_chunks = input_size + output_size
        
        self.shuffle = shuffle
        self.verbose = verbose
        
        if not shuffle:
            logging.warning('Batch size will be ignored (shuffle=False). '
                            'All constructed windows will be used to train.')
        
        # Dataloader protections
        assert self.batch_size % self.n_series_per_batch == 0, (
            f'batch_size {self.batch_size} must be multiple of '
            f'n_series_per_batch {self.n_series_per_batch}'
        )
        assert self.n_series_per_batch <= self.ts_dataset.n_series, (
            f'n_series_per_batch {n_series_per_batch} needs '
            f'to be smaller than n_series {self.ts_dataset.n_series}'
        )
                
        # Defining windows attributes by model
        self.windows_size: int
        self.padding: Tuple[int, int]
            
        self._define_attributes_by_model()
        
        # Defining sampleable time series
        self.sampleable_ts_idxs: np.ndarray
        self.n_sampleable_ts: int
            
        self._define_sampleable_ts_idxs()
        
         # Loader iterations attributes
        self.n_batches = int(np.ceil(self.n_sampleable_ts / self.n_series_per_batch)) # Must be multiple of batch_size for paralel gpu        


In [ ]:
#export
@patch
def _define_attributes_by_model(self: TimeSeriesLoader):
    if self.model in ['nbeats']:
        self.windows_size = self.input_size + self.output_size
        self.padding = (self.input_size, self.output_size)
    elif self.model in ['esrnn']:
        self.windows_size = self.len_sample_chunks
        self.padding = (0, 0)
    else:
        raise Exception(f'There is no batch strategy for {self.model}')

In [ ]:
#export
@patch
def _define_sampleable_ts_idxs(self: TimeSeriesLoader):
    sum_sample_mask = self.ts_dataset.ts_tensor[:, self.t_cols.index('sample_mask')] \
                              .sum(axis=1)
    if self.complete_inputs:
        min_mask = self.windows_size
    else:
        min_mask = self.output_size
    self.sampleable_ts_idxs = np.argwhere(sum_sample_mask > min_mask).reshape(1, -1)[0]
    self.n_sampleable_ts = self.sampleable_ts_idxs.size

In [ ]:
#export
@patch
def _get_sampleable_windows_idxs(self: TimeSeriesLoader, 
                                 ts_windows_flatten: t.Tensor) -> np.ndarray:
    """Gets indexes of windows that fulfills conditions.
    
    Parameters
    ----------
    ts_windows_flatten: t.Tensor
        Tensor of shape (windows, n_channels, windows_size)
    
    Returns
    -------
    Numpy array of indexes of ts_windows_flatten that 
    fulfills conditions.
    
    Notes
    -----
    [1] If complete_input=True
    return all windows which its ouput_size 
    has complete sample_mask and its input_size
    has complete available_mask.
    [2] If complete_input=False
    returns all windows which its
    output_size has complete sample_mask. 
    This avoids leakage.
    """
    sample_condition = t.sum(ts_windows_flatten[:, self.t_cols.index('sample_mask'), -self.output_size:], axis=1)
    sample_condition = (sample_condition == self.output_size) * 1
    if self.complete_inputs:
        available_condition = t.sum(ts_windows_flatten[:, self.t_cols.index('available_mask'), :-self.output_size], axis=1)
        available_condition = (available_condition == self.windows_size - self.output_size) * 1
        sampling_idx = t.nonzero(available_condition * sample_condition > 0)
    else:
        sampling_idx = t.nonzero(sample_condition)

    sampling_idx = sampling_idx.flatten().numpy()
    assert sampling_idx.size > 0, (
        'Check the data and masks as sample_idxs are empty, '
        'check window_sampling_limit, input_size, output_size, masks'
    )
    
    return sampling_idx

In [ ]:
#export
@patch
def _create_windows_tensor(self: TimeSeriesLoader, 
                           index: Optional[np.ndarray] = None) -> Tuple[t.Tensor, 
                                                                        t.Tensor,
                                                                        t.Tensor]:
    """Creates windows of size windows_size from
    the ts_tensor of the TimeSeriesDataset filtered by
    window_sampling_limit and ts_idxs. The step of each window
    is defined by idx_to_sample_freq.
    
    Parameters
    ----------
    index: Optional[np.ndarray]
        Indexes of time series to consider.
        Default None: returns all ts.
    
    Returns
    -------
    Tuple of three elements:
        - Windows tensor of shape (windows, channels, input_size + output_size)
        - Static variables tensor of shape (windows * series, n_static)
        - Time Series indexes for each window.
    """
    # Default ts_idxs=ts_idxs sends all the data, otherwise filters series      
    tensor, _ = self.ts_dataset \
                    .get_filtered_ts_tensor(output_size=self.output_size,
                                            window_sampling_limit=self.window_sampling_limit,
                                            ts_idxs=index)
    tensor = t.Tensor(tensor)

    padder = t.nn.ConstantPad1d(padding=self.padding, value=0)
    tensor = padder(tensor)

    # Creating rolling windows and 'flattens' them
    windows = tensor.unfold(dimension=-1, 
                            size=self.windows_size, 
                            step=self.idx_to_sample_freq)
    # n_serie, n_channel, n_time, window_size -> n_serie, n_time, n_channel, window_size
    windows = windows.permute(0, 2, 1, 3)
    windows = windows.reshape(-1, self.ts_dataset.n_channels, self.windows_size)
    
    # Broadcast s_matrix: This works because unfold in windows_tensor, orders: serie, time
    s_matrix = self.ts_dataset.s_matrix[index]
    n_ts = self.ts_dataset.n_series if index is None else len(index)
    windows_per_serie = len(windows) / n_ts
    s_matrix = s_matrix.repeat(repeats=windows_per_serie, axis=0)
    ts_idxs = index.repeat(repeats=windows_per_serie)
    
    s_matrix = t.Tensor(s_matrix)
    ts_idxs = t.as_tensor(ts_idxs, dtype=t.long)


    return windows, s_matrix, ts_idxs

In [ ]:
#export
@patch
def _windows_batch(self: TimeSeriesLoader, 
                   index: np.ndarray) -> Dict[str, t.Tensor]:
    """Creates batch based on index.
    
    Parameters
    ----------
    index: np.ndarray
        Indexes of time series to consider.
    
    Returns
    -------
    Dictionary with keys:
        - S
        - Y
        - X
        - available_mask
        - sample_mask
        - idxs
    """

    # Create windows for each sampled ts and sample random unmasked windows from each ts
    windows, s_matrix, ts_idxs = self._create_windows_tensor(index=index)
    sampleable_windows = self._get_sampleable_windows_idxs(ts_windows_flatten=windows)

    # Get sample windows_idxs of batch
    if self.shuffle:
        windows_idxs = np.random.choice(sampleable_windows, self.batch_size, replace=True)
    else:
        windows_idxs = sampleable_windows

    # Index the windows and s_matrix tensors of batch
    windows = windows[windows_idxs]
    S = s_matrix[windows_idxs]
    ts_idxs = ts_idxs[windows_idxs]

    # Parse windows to elements of batch
    Y = windows[:, self.t_cols.index('y'), :]
    X = windows[:, (self.t_cols.index('y') + 1):self.t_cols.index('available_mask'), :]
    available_mask = windows[:, self.t_cols.index('available_mask'), :]
    sample_mask = windows[:, self.t_cols.index('sample_mask'), :]

    batch = {'S': S, 'Y': Y, 'X': X,
             'available_mask': available_mask,
             'sample_mask': sample_mask,
             'idxs': ts_idxs}
    
    return batch

In [ ]:
#export
@patch
def __getitem__(self: TimeSeriesLoader, 
                index: Union[Collection[int], np.ndarray]) -> Dict[str, t.Tensor]:
    """Gets batch based on index.
    
    Parameters
    ----------
    index: Collection[int]
        Indexes of time series to consider.
    
    Returns
    -------
    Batch corresponding to index.
    """
    
    return self._windows_batch(index=np.array(index))

In [ ]:
#export
@patch
def __iter__(self: TimeSeriesLoader) -> Dict[str, t.Tensor]:
    """Batch iterator."""
    # Hierarchical sampling
    # 1. Sampling series
    if self.shuffle:
        sample_idxs = np.random.choice(a=self.sampleable_ts_idxs, 
                                       size=self.n_sampleable_ts, 
                                       replace=False)
    else:
        sample_idxs = np.array(self.sampleable_ts_idxs)

    for idx in range(self.n_batches):
        ts_idxs = sample_idxs[(idx * self.n_series_per_batch) : (idx + 1) * self.n_series_per_batch]
        # 2. Sampling windows
        batch = self[ts_idxs]
        yield batch

In [ ]:
#export
@patch
def get_n_variables(self: TimeSeriesLoader) -> Tuple[int, int]:
    """Gets number of exogenous and static variables."""
    return self.ts_dataset.n_x, self.ts_dataset.n_s

@patch
def get_n_series(self: TimeSeriesLoader) -> int:
    """Gets number of time series."""
    return self.ts_dataset.n_series

@patch
def get_max_len(self: TimeSeriesLoader) -> int:
    """Gets max len of time series."""
    return self.ts_dataset.max_len

@patch
def get_n_channels(self: TimeSeriesLoader) -> int:
    """Gets number of channels considered."""
    return self.ts_dataset.n_channels

@patch
def get_frequency(self: TimeSeriesLoader) -> str:
    """Gets infered frequency."""
    return self.ts_dataset.frequency

## Check for TimeSeriesLoader's methods

### Sampleable ts idxs

In [ ]:
from nixtla.data.utils import create_synthetic_tsdata

n_ts = 64
Y_df, X_df, S_df = create_synthetic_tsdata(n_ts=n_ts, sort=True)
ds_in_test = 10
is_test = False
batch_size = 8
len_sample_chunks = 20 #only for ESRNN

dataset = TimeSeriesDataset(Y_df=Y_df, X_df=X_df, S_df=S_df,
                            ds_in_test=ds_in_test, 
                            is_test=is_test)

In [ ]:
def test_sampleable_ts(dataset, model, input_size, output_size, 
                       len_sample_chunks,
                       complete_inputs, ds_in_test):
    """This function checks that the sample conditions of the loader 
    match the expected ones.
    Assumptions for the test:
    - Full sample mask.
    - Positive time series.
    These assumptions are satisfied by the function create_synthetic_tsdata.
    """
    loader = TimeSeriesLoader(ts_dataset=dataset,
                              model=model,
                              window_sampling_limit=dataset.max_len,
                              input_size=input_size,
                              output_size=output_size,
                              idx_to_sample_freq=1,
                              batch_size=batch_size,
                              complete_inputs=complete_inputs,
                              len_sample_chunks=len_sample_chunks,
                              shuffle=True)
    
    sizes = dataset.len_series
    if complete_inputs:
        if model in ['esrnn']:
            min_size = len_sample_chunks or input_size + output_size
        else:
            min_size = input_size + output_size
    else:
        min_size = output_size
    e_sampleable_ts_idxs = np.where(sizes > min_size + ds_in_test)[0]
    
    assert np.array_equal(loader.sampleable_ts_idxs, e_sampleable_ts_idxs)

In [ ]:
test_sampleable_ts(dataset=dataset, 
                   model='esrnn', input_size=5, output_size=5,
                   len_sample_chunks=None, complete_inputs=False, 
                   ds_in_test=ds_in_test)

In [ ]:
test_sampleable_ts(dataset=dataset, 
                   model='esrnn', input_size=5, output_size=5,
                   len_sample_chunks=len_sample_chunks, complete_inputs=True, 
                   ds_in_test=ds_in_test)

In [ ]:
test_sampleable_ts(dataset=dataset, 
                   model='nbeats', input_size=5, output_size=5,
                   len_sample_chunks=None, complete_inputs=True, 
                   ds_in_test=ds_in_test)

In [ ]:
test_sampleable_ts(dataset=dataset, 
                   model='nbeats', input_size=5, output_size=5,
                   len_sample_chunks=None, complete_inputs=False, 
                   ds_in_test=ds_in_test)

The following test checks failure for not sampleable time series.

In [ ]:
loader = TimeSeriesLoader(ts_dataset=dataset,
                          model='esrnn',
                          window_sampling_limit=dataset.max_len,
                          input_size=5,
                          output_size=5,
                          idx_to_sample_freq=1,
                          batch_size=8,
                          complete_inputs=False,
                          len_sample_chunks=15,
                          shuffle=True)

In [ ]:
def _fail_not_sampleable_ts(): return loader[[1]]
test_fail(_fail_not_sampleable_ts)

### Batches size

Tests for `__iter__` method.

The following test checks the correct size of batches. Note that the data loader only returns batches of ` batch_size` when suffle is True.
For `nbeats` the windows size should be `input_size + output_size`. Meanwhile for `esrnn` the windows size should be `len_sample_chunks`. When `len_sample_chunks=None` then `len_sample_chunks=input_size + output_size`.

In [ ]:
from nixtla.data.utils import create_synthetic_tsdata

n_ts = 64
Y_df, X_df, S_df = create_synthetic_tsdata(n_ts=n_ts, sort=True)
ds_in_test = 10
is_test = False
batch_size = 8
len_sample_chunks = 20 #only for ESRNN

dataset = TimeSeriesDataset(Y_df=Y_df, X_df=X_df, S_df=S_df,
                            ds_in_test=ds_in_test, 
                            is_test=is_test)

In [ ]:
def test_batch_size(dataset, model, input_size, output_size, len_sample_chunks,
                    expected_window_size):
    """This test checks the correct size of batches."""
    loader = TimeSeriesLoader(ts_dataset=dataset,
                              model=model,
                              window_sampling_limit=dataset.max_len,
                              input_size=input_size,
                              output_size=output_size,
                              idx_to_sample_freq=1,
                              batch_size=batch_size,
                              complete_inputs=False,
                              len_sample_chunks=len_sample_chunks,
                              shuffle=True)
    for batch in loader:
        S = batch['S']
        Y = batch['Y']
        X = batch['X']
        available_mask = batch['available_mask']
        sample_mask = batch['sample_mask']
        idxs = batch['idxs']

        test_eq(S.shape, (batch_size, dataset.n_s))
        test_eq(Y.shape, (batch_size, expected_window_size))
        test_eq(X.shape, (batch_size, dataset.n_x, expected_window_size))
        test_eq(available_mask.shape, (batch_size, expected_window_size))
        test_eq(sample_mask.shape, (batch_size, expected_window_size))
        test_eq(idxs.shape, (batch_size,))

In [ ]:
def test_batch_iteration(dataset, model, input_size, output_size, 
                         len_sample_chunks, complete_inputs,
                         shuffle):
    """This test checks proper iteration when batch_size=1."""
    loader = TimeSeriesLoader(ts_dataset=dataset,
                              model=model,
                              window_sampling_limit=dataset.max_len,
                              input_size=input_size,
                              output_size=output_size,
                              idx_to_sample_freq=1,
                              batch_size=1,
                              complete_inputs=complete_inputs,
                              len_sample_chunks=len_sample_chunks,
                              shuffle=shuffle)
    
    batches = [batch for batch in loader]
    assert len(batches) == loader.n_sampleable_ts, (
        'Expected and actual batches are different '
        'for batch_size=1'
    )

#### ESRNN

In [ ]:
len_sample_chunks = 20
test_batch_size(dataset, 'esrnn', input_size=5, output_size=5, 
                len_sample_chunks=len_sample_chunks, 
                expected_window_size=len_sample_chunks)

In [ ]:
test_batch_iteration(dataset, 'esrnn', input_size=5, output_size=5,
                     len_sample_chunks=len_sample_chunks,
                     complete_inputs=True, shuffle=True)

In [ ]:
test_batch_iteration(dataset, 'esrnn', input_size=5, output_size=5,
                     len_sample_chunks=None,
                     complete_inputs=True, shuffle=True)

In [ ]:
test_batch_iteration(dataset, 'esrnn', input_size=5, output_size=5,
                     len_sample_chunks=None,
                     complete_inputs=False, shuffle=True)

In [ ]:
test_batch_iteration(dataset, 'esrnn', input_size=5, output_size=5,
                     len_sample_chunks=None,
                     complete_inputs=False, shuffle=False)

#### NBEATS

In [ ]:
input_size = 10
output_size = 5
test_batch_size(dataset, 'nbeats', input_size=input_size, output_size=output_size, 
                len_sample_chunks=20, expected_window_size=input_size + output_size)

In [ ]:
test_batch_iteration(dataset, 'nbeats', input_size=5, output_size=5,
                     len_sample_chunks=None,
                     complete_inputs=True, shuffle=True)

In [ ]:
test_batch_iteration(dataset, 'nbeats', input_size=5, output_size=5,
                     len_sample_chunks=None,
                     complete_inputs=False, shuffle=True)

In [ ]:
test_batch_iteration(dataset, 'nbeats', input_size=5, output_size=5,
                     len_sample_chunks=None,
                     complete_inputs=False, shuffle=False)

### Proper batch construction

Tests for `__getitem__` method.

In [ ]:
from nixtla.data.utils import create_synthetic_tsdata

n_ts = 64
Y_df, X_df, S_df = create_synthetic_tsdata(n_ts=n_ts, sort=True)
ds_in_test = 5
is_test = False

dataset = TimeSeriesDataset(Y_df=Y_df, X_df=X_df, S_df=S_df,
                            ds_in_test=ds_in_test, 
                            is_test=is_test)

In [ ]:
from numpy.lib.stride_tricks import sliding_window_view

# This test only works for the dataset constructed
# before and for the 21 time series
def test_batch_construction(dataset, model, input_size, output_size, 
                            len_sample_chunks, window_step, 
                            complete_inputs, ds_in_test):
    """Final test to verify that the batch (of windows) is well constructed."""
    
    wsl = dataset.max_len
    
    if model == 'nbeats':
        windows_size = input_size + output_size
    elif model == 'esrnn':
        windows_size = len_sample_chunks or input_size + output_size

    loader = TimeSeriesLoader(ts_dataset=dataset,
                              model=model,
                              window_sampling_limit=wsl,
                              input_size=input_size,
                              output_size=output_size,
                              idx_to_sample_freq=window_step,
                              batch_size=1,
                              complete_inputs=complete_inputs,
                              len_sample_chunks=len_sample_chunks,
                              shuffle=False)
    windows = loader[[20]]['Y']
    
    #Expected windows
    uid = Y_df['unique_id'].unique()[20]
    Y_original = Y_df.query('unique_id == @uid')['y'].values
    size = Y_original.size
    Y_sample = np.zeros(wsl)
    Y_sample[-size:] = Y_original
    if model == 'nbeats':
        Y_sample = np.pad(Y_sample, (input_size, output_size))
        
    e_windows = sliding_window_view(Y_sample, window_shape=windows_size)
    if model == 'nbeats':
        e_windows = e_windows[0:-(ds_in_test + output_size):window_step]
    elif model == 'esrnn':
        e_windows = e_windows[0:-ds_in_test:window_step]
    # This test works assuming there are no series with values of zero.
    if complete_inputs:
        sampleable_windows_idxs = np.where((e_windows > 0).sum(1) == windows_size)[0]
    else:
        sampleable_windows_idxs = np.where((e_windows > 0).sum(1) >= ds_in_test)[0]
        
    e_windows = e_windows[sampleable_windows_idxs]
    
    #Comparison
    assert np.array_equal(windows, e_windows), (
        'Expected and actual windows are different'
    )

In [ ]:
test_batch_construction(dataset=dataset, 
                        model='nbeats', input_size=5, output_size=5, 
                        len_sample_chunks=None, window_step=2,
                        complete_inputs=False,
                        ds_in_test=ds_in_test)

In [ ]:
test_batch_construction(dataset=dataset, 
                        model='esrnn', input_size=5, output_size=5, 
                        len_sample_chunks=15, window_step=2,
                        complete_inputs=False, ds_in_test=ds_in_test)

In [ ]:
test_batch_construction(dataset=dataset, 
                        model='esrnn', input_size=5, output_size=5, 
                        len_sample_chunks=15, window_step=2,
                        complete_inputs=True, ds_in_test=ds_in_test)

In [ ]:
test_batch_construction(dataset=dataset, 
                        model='nbeats', input_size=5, output_size=5, 
                        len_sample_chunks=None, window_step=3,
                        complete_inputs=True,
                        ds_in_test=ds_in_test)